In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput
from typing import Dict, List, Optional
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, Field, EmailStr
import re


In [ ]:
load_dotenv(override=True)

# API Keys
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

# Print API key status
for key, name in [
    (openai_api_key, 'OpenAI'),
    (google_api_key, 'Google'),
    (deepseek_api_key, 'DeepSeek'),
    (groq_api_key, 'Groq'),
    (anthropic_api_key, 'Anthropic')
]:
    if key:
        print(f"{name} API Key exists and begins with {key[:4]}")
    else:
        print(f"{name} API Key not set (optional except OpenAI)")


In [3]:
class EmailContent(BaseModel):
    subject: str = Field(..., description="The email subject line")
    body: str = Field(..., description="The main body of the email")
    sender_title: str = Field(..., description="The professional title of the sender")
    recipient_title: str = Field(..., description="The title/role of the recipient")
    
class EmailValidation(BaseModel):
    contains_spam_words: bool = Field(..., description="Whether the email contains common spam trigger words")
    length_appropriate: bool = Field(..., description="Whether the email length is appropriate (not too short or long)")
    has_call_to_action: bool = Field(..., description="Whether the email has a clear call to action")
    spam_words_found: List[str] = Field(default_factory=list, description="List of spam words found in the email")
    
class PersonalInfoCheck(BaseModel):
    contains_personal_info: bool = Field(..., description="Whether the message contains personal information")
    info_type: Optional[str] = Field(None, description="Type of personal information found (name, email, phone, etc.)")
    found_text: Optional[str] = Field(None, description="The actual personal information found")


In [4]:
# API Base URLs
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1"

# Initialize clients
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
anthropic_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)

# Initialize models
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
claude_model = OpenAIChatCompletionsModel(model="claude-3-sonnet", openai_client=anthropic_client)


In [5]:
# Input guardrail for personal information
personal_info_checker = Agent(
    name="Personal Info Checker",
    instructions="Check if the message contains any personal information like names, emails, or phone numbers.",
    output_type=PersonalInfoCheck,
    model="gpt-4o-mini"
)

@input_guardrail
async def check_personal_info(ctx, agent, message):
    result = await Runner.run(personal_info_checker, message, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info={"personal_info": result.final_output},
        tripwire_triggered=result.final_output.contains_personal_info
    )

# Email validation agent
email_validator = Agent(
    name="Email Validator",
    instructions="Validate the email content for spam words, appropriate length, and presence of call to action.",
    output_type=EmailValidation,
    model="gpt-4o-mini"
)

@output_guardrail
async def validate_email(ctx, agent, output):
    validation_result = await Runner.run(email_validator, output, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info={"validation": validation_result.final_output},
        tripwire_triggered=validation_result.final_output.contains_spam_words or not validation_result.final_output.length_appropriate
    )

# HTML email conversion function
@function_tool
def convert_to_html(email_content: EmailContent) -> str:
    """Convert the email content to HTML format with professional styling"""
    html = f"""
    <!DOCTYPE html>
    <html>
    <body style="font-family: Arial, sans-serif; line-height: 1.6; max-width: 600px; margin: 0 auto;">
        <p>Dear {email_content.recipient_title},</p>
        {email_content.body}
        <p>Best regards,<br>
        {email_content.sender_title}</p>
    </body>
    </html>
    """
    return html

# SendGrid email sending function
@function_tool
def send_email(email_content: EmailContent, html_body: str) -> Dict[str, str]:
    """Send the email using SendGrid"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sanders.brett@gmail.com")
    to_email = To("blogbrett@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, email_content.subject, content)
    response = sg.client.mail.send.post(request_body=mail.get())
    return {"status": "success"}


In [6]:
# Sales agent instructions with different personalities
professional_instructions = """You are a professional sales agent for ComplAI, a company providing AI-powered SOC2 compliance and audit preparation tools.
You write structured, formal emails that emphasize our expertise and track record. Your output should be an EmailContent object."""

casual_instructions = """You are a friendly, approachable sales agent for ComplAI, a company providing AI-powered SOC2 compliance and audit preparation tools.
You write conversational emails that build rapport while maintaining professionalism. Your output should be an EmailContent object."""

direct_instructions = """You are a results-focused sales agent for ComplAI, a company providing AI-powered SOC2 compliance and audit preparation tools.
You write concise, value-proposition focused emails that get straight to the point. Your output should be an EmailContent object."""

# Create sales agents with different models and personalities
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=professional_instructions,
    model=deepseek_model,
    output_type=EmailContent
)

sales_agent2 = Agent(
    name="Casual Sales Agent",
    instructions=casual_instructions,
    model=gemini_model,
    output_type=EmailContent
)

sales_agent3 = Agent(
    name="Direct Sales Agent",
    instructions=direct_instructions,
    model=llama3_3_model,
    output_type=EmailContent
)

# Create tools from the agents
tool1 = sales_agent1.as_tool(tool_name="professional_email", tool_description="Generate a professional, formal sales email")
tool2 = sales_agent2.as_tool(tool_name="casual_email", tool_description="Generate a friendly, conversational sales email")
tool3 = sales_agent3.as_tool(tool_name="direct_email", tool_description="Generate a concise, direct sales email")


In [7]:
email_manager_instructions = """You are an enhanced email manager responsible for processing and sending sales emails.
Your process:
1. Receive an EmailContent object from a sales agent
2. Convert it to HTML using the convert_to_html tool
3. Send the email using the send_email tool
4. Report back on the success of the operation

You ensure all emails meet our quality standards and contain necessary components."""

email_manager = Agent(
    name="Enhanced Email Manager",
    instructions=email_manager_instructions,
    tools=[convert_to_html, send_email],
    model="gpt-4o-mini"
)

# Create the sales manager with enhanced guardrails
sales_manager_instructions = """You are a strategic sales manager for ComplAI.
Your responsibilities:
1. Use all three sales agent tools to generate different email versions
2. Evaluate each email for effectiveness and appropriateness
3. Select the most effective email based on:
   - Clear value proposition
   - Professional tone
   - Compelling call to action
4. Hand off the selected email to the Email Manager for processing

Never generate emails yourself - always use the provided tools."""

sales_manager = Agent(
    name="Strategic Sales Manager",
    instructions=sales_manager_instructions,
    tools=[tool1, tool2, tool3],
    handoffs=[email_manager],
    model="gpt-4o-mini",
    input_guardrails=[check_personal_info],
    output_guardrails=[validate_email]
)


In [ ]:
# Test messages
test_messages = [
    "Send a cold sales email to the CTO about our SOC2 compliance automation",
    "Write a sales email to Alice Smith about our audit preparation tools",  # Should trigger personal info guardrail
    "Send an email to the Head of Security explaining our AI-powered compliance monitoring"
]

# Run tests with tracing
async def run_tests():
    for message in test_messages:
        print(f"\nTesting message: {message}")
        print("-" * 50)
        
        with trace(f"Enhanced Sales Email System - {message[:30]}"):
            try:
                result = await Runner.run(sales_manager, message)
                print("Email generation completed successfully")
            except Exception as e:
                print(f"Error occurred: {str(e)}")
        print("-" * 50)

# Run the tests
await run_tests()
